In [ ]:
%%bash
set -euo pipefail
cd "$(dirname "$(dirname "$PWD")")"
source scripts/bin/setup_project_env.sh
ssh -o StrictHostKeyChecking=accept-new bastion-host 'true'
ssh -o StrictHostKeyChecking=accept-new swarm-manager 'hostname'
docker info --format 'Name: {{.Name}} | Swarm: {{.Swarm.LocalNodeState}}'


# Deploy Monitoring (Prometheus/Grafana) and Logging (Loki/Promtail)

### 사전 준비
1. `source scripts/bin/setup_project_env.sh`
2. `export DOCKER_HOST="ssh://swarm-manager"`
3. 필요 시 포트 포워딩: `ssh -N -L 9090:localhost:9090 -L 3000:localhost:3000 -L 3100:localhost:3100 swarm-manager`

### 1. 네트워크 생성

In [ ]:
%%bash
export DOCKER_HOST="ssh://swarm-manager"
docker network create --driver overlay monitoring_net 2>/dev/null || true
docker network create --driver overlay logging_net 2>/dev/null || true

### 2. 모니터링 스택 배포 (Prometheus + Grafana)

In [ ]:
%%bash
set -euo pipefail
cd "$(dirname "$(dirname "$PWD")")"
source scripts/bin/setup_project_env.sh
STACK_FILE="02-operations/monitoring/stack.yml"
docker stack deploy -c "$STACK_FILE" monitoring


### 3. 로깅 스택 배포 (Loki + Promtail)

In [ ]:
%%bash
export DOCKER_HOST="ssh://swarm-manager"
STACK_FILE=../02-operations/logging/stack.yml
docker stack deploy -c ${STACK_FILE} logging


### 4. 서비스 확인

In [ ]:
%%bash
export DOCKER_HOST="ssh://swarm-manager"
docker service ls | grep monitoring
docker service ls | grep logging

### 5. 접속 정보
- Prometheus: http://localhost:9090
- Grafana: http://localhost:3000 (ID/PW: admin/admin)
- Loki API: http://localhost:3100
스택 제거: `docker stack rm monitoring`, `docker stack rm logging`